<a href="https://colab.research.google.com/github/JaebinChoi/Recommendation-System/blob/master/content_based_recommendation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

path = '/gdrive/My Drive/Recommend'

### Read Data

In [ ]:
movies = pd.read_csv('%s/data/ml-latest-small/movies.csv' % path)
ratings = pd.read_pickle('%s/data/ml-latest-small/ratings_updated.p' % path)
genres = pd.read_pickle('%s/data/ml-latest-small/genres.p' % path)

In [ ]:
ratings.sample()

,userId,movieId,rating,timestamp
88939,573,69122,4.5,2009-07-29 03:55:41


In [ ]:
genres.sample()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
65514,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


### Preprocessing

In [ ]:
# 왼쪽 테이블은 movieId 를 기준으로, 오른쪽 테이블은 인덱스를 기준으로 조인
ratings = ratings.merge(genres, left_on='movieId', right_index=True)
ratings.sample()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
5295,35,595,3.0,1996-05-01 08:30:38,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0


In [ ]:
# 데이터에 0 을 NaN으로
ratings = ratings.replace(0, np.nan)
ratings.sample()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
53610,354,586,3.0,2008-01-20 23:09:50,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(ratings, random_state=42, test_size=0.1)

In [ ]:
print(train.shape)
print(test.shape)

(90777, 24)
(10087, 24)


### Item Profile

In [ ]:
# Item Profile : 영화를 설명할 수 있는 데이터
# 해리포터와 마법사의 돌
genres.loc[4896]

(no genres listed)    0
Action                0
Adventure             1
Animation             0
Children              1
Comedy                0
Crime                 0
Documentary           0
Drama                 0
Fantasy               1
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               0
Sci-Fi                0
Thriller              0
War                   0
Western               0
Name: 4896, dtype: int64

### User Profile

In [ ]:
genre_cols = genres.columns
genre_cols

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [ ]:
# 해당 장르에 평점을 넣어서 반환
for cols in genre_cols:
  train[cols] = train[cols] * train['rating']

In [ ]:
train.head()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
85575,555,2111,3.0,2001-01-06 23:04:09,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53393,352,68954,4.0,2017-05-01 21:27:54,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69018,448,2378,4.5,2006-11-27 10:09:27,NaN,NaN,NaN,NaN,NaN,4.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91506,594,140,4.5,2005-02-21 08:14:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN
79967,500,2539,4.0,2001-11-12 01:20:39,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 각 유저 별로 Action 아이템에 준 평점의 평균
# train.groupby('userId')['Action'].mean()

userId
1       4.294872
2       3.888889
3       3.333333
4       3.454545
5       3.111111
          ...   
607     3.650000
608     3.316000
609     3.090909
610     3.608511
1000    3.946667
Name: Action, Length: 611, dtype: float64

In [ ]:
user_profile = train.groupby('userId')[genre_cols].mean()
user_profile.sample()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
116,NaN,3.732143,3.5,3.5,3.125,3.403226,3.055556,NaN,3.147059,3.222222,NaN,3.071429,1.0,4.0,3.8,3.533333,3.625,3.5,3.0,4.5


### My User Profile

In [ ]:
user_profile.loc[1000]

(no genres listed)         NaN
Action                3.946667
Adventure             3.992308
Animation             4.250000
Children              4.000000
Comedy                4.333333
Crime                 2.500000
Documentary                NaN
Drama                 4.300000
Fantasy               4.125000
Film-Noir                  NaN
Horror                4.000000
IMAX                  3.500000
Musical               4.000000
Mystery                    NaN
Romance               4.100000
Sci-Fi                4.030769
Thriller              3.160000
War                        NaN
Western                    NaN
Name: 1000, dtype: float64

## Predict Rating of Movie (Sample Data)

In [ ]:
sample = test.loc[2227]
sample_user = sample['userId']
sample_user_profile = user_profile.loc[sample_user]
sample_user_profile

#len(test[test['userId'] == sample['userId']]) # test 데이터에서 2227번의 userId 개수

(no genres listed)         NaN
Action                3.583770
Adventure             3.632576
Animation             3.732143
Children              3.500000
Comedy                3.492481
Crime                 3.895833
Documentary           3.937500
Drama                 3.894330
Fantasy               3.642857
Film-Noir             3.875000
Horror                3.230769
IMAX                  3.676471
Musical               3.700000
Mystery               4.009615
Romance               3.666667
Sci-Fi                3.636364
Thriller              3.769737
War                   4.000000
Western               4.181818
Name: 18, dtype: float64

In [ ]:
print(sample['movieId'])
print()
print(movies[movies['movieId'] == 114066])
print()

sample[genre_cols]

114066

      movieId                        title                     genres
8510   114066  20,000 Days on Earth (2014)  Documentary|Drama|Musical



(no genres listed)    NaN
Action                NaN
Adventure             NaN
Animation             NaN
Children              NaN
Comedy                NaN
Crime                 NaN
Documentary             1
Drama                   1
Fantasy               NaN
Film-Noir             NaN
Horror                NaN
IMAX                  NaN
Musical                 1
Mystery               NaN
Romance               NaN
Sci-Fi                NaN
Thriller              NaN
War                   NaN
Western               NaN
Name: 2227, dtype: object

In [ ]:
(sample_user_profile * sample[genre_cols]).mean()

3.8439432989690716

### 전체 데이터로 확장

In [ ]:
# for 진행상황 표시
from tqdm import tqdm_notebook

In [ ]:
# 참고사항
# for idx, row in test.iterrows():
#   print('idx : ' + str(idx))
#   print(row)
#   break

predict = []
for idx, row in test.iterrows():
  user = row['userId']
  # user profile * itme profile
  predict.append((user_profile.loc[user] * row[genre_cols]).mean())

In [ ]:
test['predict'] = predict

# NaN 인 경우 : 유저가 해당 장르의 영화를 평가한 적이 없다.
# 해당 유저의 평점의 평균으로 예측
test.loc[test['predict'].isnull(), 'predict'] = train['rating'].mean()

In [ ]:
test[test['predict'].isnull()]

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,predict


### Model Validation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(test['rating'], test['predict'])
rmse = np.sqrt(mse)
rmse

0.9196772254138662